In [1]:
# Resources
# https://machinelearningmastery.com/greedy-layer-wise-pretraining-tutorial/
# https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/
# https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html#sphx-glr-auto-examples-classification-plot-digits-classification-py
# https://towardsdatascience.com/a-simple-2d-cnn-for-mnist-digit-recognition-a998dbc1e79a

# Standard scientific Python imports
import tensorflow.python.keras as k
from tensorflow.python.keras.datasets import mnist
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.core import Dropout, Flatten, Activation
from tensorflow.python.keras.layers import Dense, Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
import time
import os
import cv2
import numpy as np

def simpleEx(digits):
    # The data that we are interested in is made of 8x8 images of digits, let's
    # have a look at the first 4 images, stored in the `images` attribute of the
    # dataset.  If we were working from image files, we could load them using
    # matplotlib.pyplot.imread.  Note that each image must have the same size. For these
    # images, we know which digit they represent: it is given in the 'target' of
    # the dataset.
    _, axes = plt.subplots(2, 4)
    images_and_labels = list(zip(digits.images, digits.target))
    for ax, (image, label) in zip(axes[0, :], images_and_labels[:4]):
        ax.set_axis_off()
        ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
        ax.set_title('Training: %i' % label)

    # To apply a classifier on this data, we need to flatten the image, to
    # turn the data in a (samples, feature) matrix:
    n_samples = len(digits.images)
    data = digits.images.reshape((n_samples, -1))

    # Create a classifier: a support vector classifier
    # supervised learning models with associated learning algorithms 
    #that analyze data used for classification and regression analysis. 
    classifier = svm.SVC(gamma=0.001)

    # Split data into train and test subsets
    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.5, shuffle=False)

    # We learn the digits on the first half of the digits
    t0 = time.time()
    classifier.fit(X_train, y_train)
    t1 = time.time()
    total = t1 - t0
    print("Time for simpleEx ", total)

    # Now predict the value of the digit on the second half:
    predicted = classifier.predict(X_test)

    images_and_predictions = list(zip(digits.images[n_samples // 2:], predicted))
    for ax, (image, prediction) in zip(axes[1, :], images_and_predictions[:4]):
        ax.set_axis_off()
        ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
        ax.set_title('Prediction: %i' % prediction)

    print("Classification report for classifier %s:\n%s\n"
          % (classifier, metrics.classification_report(y_test, predicted)))
    # isp = metrics.plot_confusion_matrix(classifier, X_test, y_test)
    # isp.figure_.suptitle("Confusion Matrix")
    # rint("Confusion matrix:\n%s" % disp.confusion_matrix)
    # lt.show()


def reshape(X_train, X_test, y_train, y_test):
    img_rows = 100;
    img_cols = 100;
    # X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    # X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    # input_shape = (1, img_rows, img_cols)
    # else
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    # more reshaping
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    
    y_train = y_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    y_test = y_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    v# more reshaping
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    #num_category = 10
    # convert class vectors to binary class matrices
    #y_train = k.utils.to_categorical(y_train, num_category)
    #y_test = k.utils.to_categorical(y_test, num_category)
    print(X_train.shape)
    return X_train, X_test, y_train, y_test


def conv2DEx(xTrain, xTest, yTrain, yTest):
    #xTrain, xTest, yTrain, yTest = reshape(X_train, X_test, y_train, y_test)
    ##model building
    #is a linear stack of Layers. You can create a Sequential model and 
    #define all of the layers in the constructor
    model = Sequential()
    # convolutional layer with rectified linear unit activation
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(100, 100, 3)))
    # 32 convolution filters used each of size 3x3
    # again
    model.add(Conv2D(64, (3, 3), activation='relu'))
    # 64 convolution filters used each of size 3x3
    # choose the best features via pooling
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # randomly turn neurons on and off to improve convergence
    model.add(Dropout(0.25))
    # flatten since too many dimensions, we only want a classification output
    model.add(Flatten())
    # fully connected to get all relevant data
    model.add(Dense(128, activation='relu'))
    # one more dropout for convergence' sake :) 
    model.add(Dropout(0.5))
    # output a softmax to squash the matrix into output probabilities
    model.add(Dense(10, activation='softmax'))

    # Adaptive learning rate (adaDelta) is a popular form of gradient descent rivaled only by adam and adagrad
    # categorical ce since we have multiple classes (10) 
    model.compile(loss=k.losses.categorical_crossentropy,
                  optimizer=k.optimizers.Adadelta(),
                  metrics=['accuracy'])
    batch_size = 128
    num_epoch = 10
    # model training
    t0 = time.time()
    model_log = model.fit(xTrain, yTrain,
                          batch_size=batch_size,
                          epochs=num_epoch,
                          verbose=1,
                          validation_data=(xTest, yTest))
    t1 = time.time()
    total = t1 - t0
    print("Time for conv2DEx ", total)

    # define, fit and evaluate the base autoencoder



# evaluate the autoencoder as a classifier
def evaluateAutoencoder(model, xTrain, xTest, yTrain, yTest):
    batch_size = 128
    num_epoch = 10
    t0 = time.time()
    model_log = model.fit(xTrain, yTrain,
                          batch_size=batch_size,
                          epochs=num_epoch,
                          verbose=1,
                          validation_data=(xTest, yTest))
    t1 = time.time()
    total = t1 - t0
    print("Time for evaluateAutoencoder ", total)


# add one new layer and re-train only the new layer
def add_layer_to_autoencoder(model, xTrain, xTest, yTrain, yTest):
    # remember the current output layer
    output_layer = model.layers[-1]
    # remove the output layer
    model.pop()
    # mark all remaining layers as non-trainable
    for layer in model.layers:
        layer.trainable = False
    # add a new hidden layer
    model.add(Dense(128, activation='relu'))
    # re-add the output layer
    model.add(output_layer)
    return model


def base_autoencoder(xTrain, xTest, yTrain, yTest):
    model = Sequential()
    # convolutional layer with rectified linear unit activation
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(100, 100, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    # fully connected to get all relevant data
    model.add(Dense(128, activation='relu'))
    # one more dropout for convergence' sake :) 
    model.add(Dropout(0.5))
    # output a softmax to squash the matrix into output probabilities
    model.add(Dense(10, activation='softmax'))
    model.compile(loss=k.losses.categorical_crossentropy,
                  optimizer=k.optimizers.Adadelta(),
                  metrics=['accuracy'])
    batch_size = 128
    num_epoch = 10
    # model training
    t0 = time.time()
    model_log = model.fit(xTrain, yTrain,
                          batch_size=batch_size,
                          epochs=num_epoch,
                          verbose=1,
                          validation_data=(xTest, yTest))
    t1 = time.time()
    total = t1 - t0
    print("Time for baseAutoEncoder ", total)
    return model

def preTrain(X_train, y_train, X_test, y_test):
    xTrain, xTest, yTrain, yTest = reshape(X_train, X_test, y_train, y_test)
    ##model building
    model = base_autoencoder(xTrain, xTest, yTrain, yTest)
    model = add_layer_to_autoencoder(model, xTrain, xTest, yTrain, yTest)
    evaluateAutoencoder(model, xTrain, xTest, yTrain, yTest)

def getData():
    # point to my own data, will show the first one just as a sanity check
    IMG_SIZE = 100
    trainingData = []
    newResizeImg = []
    DATADIR = os.getcwd() + "\\OneDrive\\Documents\\Spring 2020\\CleanData\\training\\"
    for img in os.listdir(DATADIR):  # iterate over each image per dress and shoe
        newImg = cv2.imread(os.path.join(DATADIR, img))
        # newResizeImg = newImg
        newResizeImg = cv2.resize(newImg, (IMG_SIZE, IMG_SIZE))
        trainingData.append(newResizeImg)
        # plt.imshow(img_array)  # graph it
        # plt.show()  # display!
        # we just want one for now so break
    print(len(trainingData));
    testX = trainingData[152:202]
    testY = trainingData[101:151]
    trainX = trainingData[0:50]
    trainY = trainingData[51:100]
    testX = np.asarray(testX)
    testY = np.asarray(testY)
    trainY = np.asarray(trainY)
    trainX = np.asarray(trainX)
    return testX, testY, trainX, trainY
def main():
    testX, testY, trainX, trainY = getData()
    # The digits dataset
    #(X_train, y_train), (X_test, y_test) = mnist.load_data()
    #digits = datasets.load_digits()
    #simpleEx(digits)
    #preTrain(X_train, y_train, X_test, y_test)
    conv2DEx(testX, testY, trainX, trainY)


main()

223


ValueError: A target array with shape (50, 100, 100, 3) was passed for an output of shape (None, 10) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.